## Anydice: two-colored polyhedral dice pool

[StackExchange question.](https://rpg.stackexchange.com/questions/198579/anydice-two-colored-polyhedral-dice-pool)

1. Roll two pools of dice, not necessarily all of the same size: a "green" pool and a "red" pool.
2. Green dice and red dice that rolled the same number eliminate each other 1:1.
3. If the two highest remaining dice are green, or there are only green dice remaining, it's a full success.
4. If the two highest remaining dice are red, or there are only green dice remaining, it's a failure.
5. Otherwise (the two highest dice are of different colors, or no dice remain), it's a partial success.

In [1]:
%pip install icepool

import icepool
from icepool import d4, d6, d8, d10, d12, MultisetEvaluator

class GreenRed(MultisetEvaluator):
    def next_state(self, state, order, outcome, green, red):
        # State is the number of top-two dice for green and red.
        top_green, top_red = state or (0, 0)
        # If there are remaining places in the top two...
        remaining_top_two = 2 - (top_green + top_red)
        if remaining_top_two > 0:
            # Compute the number of non-eliminated dice that rolled this outcome.
            net = green - red
            # Then add them to the winning team's top two.
            if net > 0:
                top_green += min(net, remaining_top_two)
            elif net < 0:
                top_red += min(-net, remaining_top_two)
        return top_green, top_red
    
    def final_outcome(self, final_state, *_):
        top_green, top_red = final_state
        if (top_green > 0) and not (top_red > 0):
            return 2
        elif (top_red > 0) and not (top_green > 0):
            return 0
        else:
            return 1
    
    def order(self):
        # See outcomes in descending order.
        return -1
    
green_red = GreenRed()
# The argument lists are implicitly cast to pools.
print(green_red.evaluate([d10, d8], [d6, d8]))

Die with denominator 3840

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |      791 |  20.598958% |
|       1 |     2784 |  72.500000% |
|       2 |      265 |   6.901042% |




In [2]:
# A larger calculation.
print(green_red.evaluate([d12, d10, d8, d6, d4, d12, d10, d8, d6, d4], [d12, d10, d8, d6, d4, d12, d10, d8, d6, d4]))

Die with denominator 281792804290560000

| Outcome |          Quantity | Probability |
|--------:|------------------:|------------:|
|       0 | 93805993954176668 |  33.288996% |
|       1 | 94180816382206664 |  33.422009% |
|       2 | 93805993954176668 |  33.288996% |


